In [1]:
import pandas as pd


train_df = pd.read_parquet('./datagame-2023/label_train_source.parquet')
test_df = pd.read_parquet('./datagame-2023/label_test_source.parquet')

In [2]:
reserve = pd.concat([train_df, test_df], axis=0)

In [3]:
sample_df = pd.read_csv("./datagame-2023/sample.csv")
sample_id = sample_df[['session_id']]

# Excluded

In [4]:
same_df = pd.read_csv('./same_df.csv')
excluded_df = same_df[same_df.iloc[:, 1:].isna().any(axis=1)]
same_df = same_df[same_df.iloc[:, 1:].notna().any(axis=1)]
excluded_id  = excluded_df[['session_id']]

# Most Played

In [5]:
reserve_songs = train_df['song_id'].value_counts()

In [6]:
most_played_df = pd.read_csv("./Top5_excluded_test_datasets.csv")
mask = (most_played_df['variety'] == 20)
excluded_20 = most_played_df[mask]
most_played_df = most_played_df[~mask]
excluded_20_id = excluded_20[['session_id']]

In [7]:
mostPlayed_combined = pd.concat([most_played_df['session_id'], most_played_df.iloc[:, 6:].drop("variety", axis=1)], axis=1)
mostPlayed_combined.columns = sample_df.columns

In [8]:
mostPlayed_combined.reset_index(drop=True, inplace=True)
idx = 0
for col_idx, column in mostPlayed_combined.drop("session_id", axis=1).items():
    for row_idx, value in enumerate(column):
        if pd.isna(value):
            mostPlayed_combined.loc[row_idx, col_idx] = reserve_songs.keys()[idx]
            idx += 1


# Convert

In [9]:
def convert(df):
    songs_per_session = df.sort_values(by='listening_order').groupby("session_id").head(20)
    new_df = songs_per_session.pivot_table(index='session_id', values='song_id', columns=songs_per_session.groupby("session_id").cumcount(), aggfunc='first')
    new_df.columns = [f'song_id_{column + 1}' for column in new_df.columns]
    new_df = new_df.reset_index()
    return new_df
diverse_test, diverse_train, diverse_reserve = convert(test_df), convert(train_df), convert(reserve)

In [10]:
df_numeric = diverse_test.apply(lambda col: col.astype('category').cat.codes)


In [11]:
def process_row(row):
    curr_id = row['session_id']
    next_data = diverse_reserve.loc[diverse_reserve['session_id'] == (curr_id + 1), diverse_reserve.columns[1: 6]]
    next_data['session_id'] = curr_id
    return next_data
combined_data = excluded_20_id.apply(process_row, axis=1)
combined_data = pd.concat(combined_data.to_list(), axis=0, ignore_index=True)
combined_data

,song_id_1,song_id_2,song_id_3,song_id_4,song_id_5,session_id
0,9d438f134d1e172892a10a35d00cd48d,4a27803802d6090389507671d8aed6eb,85422f927d88358292985cb319d216fa,9d86e8ea4ee237d367bf8bb93323fae3,e07029ed777503e9427e946fcff1bd7b,8
1,51bdd277a9ee6a1443a8b5cae566e7a9,a4851048d75f816f705c257332ebd2df,3b8690eaf3cb39ea21cd6ff2b2f477b9,4db1c69e9d0c0ebad400c73904806f9e,c298182fd4b8afc24c62ef27475f1346,9
2,4ec44d5d87df923f0f2b2067ab212d56,ba684c8314c2fa4d2ef007d2e0aa0f51,faae5069712da4b1b1f5c1537dc86368,df41f9981e56e92d6e13f638f080c65d,8b14a090f88b020f1fdd9ac4fb286576,38
3,7873629643d3d71732b639e37c381d00,d72310c97fc3cb50ef5bd5811f2b1768,ce9734f7bd15ec793a8ccbf11f08eee1,4daf6a0ea40a622927fa7a8bcf71d07b,febdbc34c0543eb8c2106d8831929d4d,39
4,4ea1d746b3c657e7260f7f94783d481f,95e2ced9d30a30fde9ea35735da8c43d,5e230b6170b453e0b1acb3e0aa3cf041,d72310c97fc3cb50ef5bd5811f2b1768,598e8b0ec5a34fa596eca8df1e0d0b7a,49
...,...,...,...,...,...,...
56061,6428fcf28017fdf5add3fef81197525b,663783403c0e3aabec9fb46e5b7e91a9,541b72a2c4bbd61c499be4b0780b4e68,d5fca7c8ed07689f45843737a995c03a,86d7d1e440c82bbe706fdbad2f0bf109,715248
56062,5ef21ae99b0ab15ac7d364b8c56a7731,0e0e08eb48fb741f136e6915831fdb32,d67a35f20d2488d0ce2621c3058b7236,4b4ea8653014d02ff611cf67d45d63d4,509db0b501f83c2972e105b7c319a86a,715259
56063,8a2fc1a4b86c6b79a6186ee6b54d79e0,af894be6f2cec95b9de30b0a128c83bd,3dbeb7f4350afdacfe4ebbf3d314a6e3,c489727635800ce9fa0a5d9f10db0c52,3dbeb7f4350afdacfe4ebbf3d314a6e3,715298
56064,733265f915bee31b0828d3ec292e57c7,8fe2d77d88b29efcb24abc3b8c7033a8,91fc38adadf43d686b4ced3ffc9f8ec7,a3a92a386ef3123788e3448f90298733,a40bcd6fc6ee237ca935ea5be909e930,715309


In [12]:
merged_df = excluded_20_id.merge(combined_data, on='session_id', how='left')
merged_df.columns = [i for i in sample_df.columns]

# Concatenation

In [13]:
concatenated_df = pd.concat([mostPlayed_combined, same_df], axis=0)

In [14]:

diverse_test[~diverse_test['session_id'].isin(concatenated_df['session_id'])].to_csv("excluded_data.csv", index=False)
concatenated_df.to_csv("included_data.csv", index=False)